# Distributed flexible

In this notebook we are showing the prototype of using flexible as a solution for training in distributed environments. Note that the public API may change a lot and right now it is limited. At the moment we can spawn a server which is able to:
- Send weights to their clients
- Collect weights from their clients
- Ask their clients to run training and or evaluation and get a metrics report

## Building the client

First we are going to show how to build a client in Flexible, in a file called `client.py` we will write the following code:

In [ ]:
from flex.distributed.client import Client
from flex.model import FlexModel
from flex.data import Dataset
import numpy as np
from typing import List


# We create a Client class which tell us how to eval, train, set and collect weights from a FlexModel
class BasicClient(Client):
    def set_weights(self, model: FlexModel, weights: List[np.ndarray]):
        # Implement here your logic for setting weights
        # Every weight we get is set to float32 for compatibility reasons
        model["model"] = weights

    def get_weights(self, model: FlexModel) -> List[np.ndarray]:
        # Implement here your logic for extracting weights
        # Every weight will be converted to float32 for compatibility reasons
        return model.get("model", [])

    def train(self, model: FlexModel, data: Dataset):
        # Implement here your training logic
        # We must return a dictionary with str as keys and float as values with the metrics
        return {"accuracy": 1.0}

    def eval(self, model: FlexModel, data: Dataset):
        # Implement here your training logic
        # We must return a dictionary with str as keys and float as values with the metrics
        return {"accuracy": 1.0}


if __name__ == "__main__":
    # Please note that we must initialize the model before creating the client
    model = FlexModel()
    dataset = Dataset.from_array([1, 2, 3], [1, 2, 3])
    # Finally run the model in order to connect to the server
    BasicClient(dataset, model, dataset).run(address="localhost:50051")

## Run the server
Now we are going to create a server and show how it works, in a file called `server.py` we will write the following code:

In [ ]:
from flex.distributed.server import Server

if __name__ == "__main__":
    from time import sleep

    server = Server()
    # Start listening
    server.run(address="localhost", port=50051)  # Listeng to port 50051
    # We can know how many clients are connected by the length of the server
    while len(server) == 0:
        print("Waiting for clients...")
        sleep(1)

    # We can also know their ids
    ids = server.get_ids()
    print(f"Ids={ids}")
    # Now we will send weights to our client!
    # We specify the ids so we only ask for those clients
    server.send_weights(np.ones((1, 1)), node_ids=ids)
    # We ask then to train
    train_metrics = server.train(node_ids=ids)
    # And inspect their metrics
    print(f"Train metrics = {train_metrics}")
    # Now we will extract their weights
    client_weights = server.collect_weights(node_ids=ids)
    # And inspect them
    print(f"Weights collected = {client_weights}")
    # Finally we stop the server
    server.stop()

And that is! It is very simple, note that we will need more boilerplate for aggregation or evaluating the model on the server but we are working on it!